## RAG System Using Llama2 With Hugging Face

Everything is this notebook works fine. But the response given by the LLM is not much useful

In [ ]:
!pip freeze

In [ ]:
!pip install pypdf

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
## Embedding
!pip install sentence_transformers

In [ ]:
!pip install llama_index

In [ ]:
!pip install llama-index-llms-huggingface

In [ ]:
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

documents=SimpleDirectoryReader("/kaggle/input/webspire").load_data()
documents

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
I have provided a whole codebase in Ruby on Rails as a text file. YOu need to answer my questions based on that
"""
## Default format supportable by LLama2
query_wrapper_prompt=PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!pip install langchain-huggingface

In [ ]:
%pip install llama-index-embeddings-langchain

In [ ]:
from llama_index.core import Settings
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

import torch

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Enable 8-bit quantization
    llm_int8_enable_fp32_cpu_offload=True  # Offload FP32 layers to CPU if needed
)

# Load the model with quantization
model_name = "tiiuae/falcon-7b-instruct"
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    model_name=model_name,
    tokenizer_name=model_name,
    device_map="auto",  # Automatically map model layers to available devices
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": quantization_config
    },
)

# Define the embedding model
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

# Configure global settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024  # Optional: Set the chunk size if needed

# Proceed with your application logic
# For example, creating an index or querying


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

index=VectorStoreIndex.from_documents(documents)

In [ ]:
index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
import gc

gc.collect()

In [ ]:
response=query_engine.query("I could see that the application has used pagination for infinite scrolling. What exact library is being used here? If so how do you say that. In which file does that library being used ?")

In [ ]:
print(response)

In [ ]:
response=query_engine.query("what is YOLO?")

In [ ]:
print(response)